# 1.ライブラリの読み込み
上から順に
* グラフを書くためにmatplotlib
* データの整理のためにpandas
* 数学的な計算のためにnumpy
* 時系列データを扱うためにdatetime
* matplotlibでdatetimeのフォーマットを変えるためにmdates

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.dates as mdates
from scipy import signal

# 2.センサーデータの読みこみと確認
ファイルの読み込みに使用した変数名の命名則については以下の通りである．

例　セッション1のAさんのセンサデータ →　data1_a
　　セッション1のAさんのスケーリングデータ　→　data1_a_label

In [2]:
%time
#sensor_dataの読み込み

data1_a = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-121953632-ags.csv', encoding="shift-jis")
data1_b = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-122231707-ags.csv', encoding="shift-jis")
data1_c = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-121953885-ags.csv', encoding="shift-jis")
data1_d = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-121954011-ags.csv', encoding="shift-jis")
data2_a = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-145759193-ags.csv', encoding="shift-jis")
data2_b = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-145733362-ags.csv', encoding="shift-jis")
data2_c = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-150018208-ags.csv', encoding="shift-jis")
data2_d = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-145806707-ags.csv', encoding="shift-jis")

#emotion_labelの読み込み

data1_a_label = pd.read_csv('emotion_label/label/emotion_A_20201111_1.csv')
data1_b_label = pd.read_csv('emotion_label/label/emotion_B_20201111_1.csv')
data1_c_label = pd.read_csv('emotion_label/label/emotion_C_20201111_1.csv')
data1_d_label = pd.read_csv('emotion_label/label/emotion_D_20201111_1.csv')
data2_a_label = pd.read_csv('emotion_label/label/emotion_A_20201111_2.csv')
data2_b_label = pd.read_csv('emotion_label/label/emotion_B_20201111_2.csv')
data2_c_label = pd.read_csv('emotion_label/label/emotion_C_20201111_2.csv')
data2_d_label = pd.read_csv('emotion_label/label/emotion_D_20201111_2.csv')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 2.62 µs


## 2.1 TSND151加速度，角速度センサデータとスケーリングデータの中身
今回のセンサデータの列はセンサの種類，時間，加速度のx，y，z，角速度x，y，zで構成されており，加速度の単位は0.1mG，角速度の単位は0.01dpsである．

スケーリングデータの列は，レコーディング動画の経過時間，感情のレベル値，発話されたユーザ名となっている．emotion_levelの感情値については今回は以下のように考えた．

-4〜-1→negative  0→neutral  １〜4→positive

In [3]:
data1_a_label

,timeHMS,emotion_level,subject
0,5:47,1,D
1,5:51,1,B
2,5:58,2,C
3,6:07,1,B
4,6:24,0,B
5,6:34,3,D
6,6:57,1,B
7,7:06,3,B
8,7:49,2,C
9,8:29,1,B


In [4]:
#生データの確認
display(data1_a)
display(data1_d_label)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z
0,ags,12:19:53.685,9428,483,4399,163,494,364
1,ags,12:19:53.690,9435,463,4382,316,391,407
2,ags,12:19:53.695,9449,451,4291,322,263,377
3,ags,12:19:53.700,9467,453,4174,181,439,328
4,ags,12:19:53.705,9481,309,3989,-25,232,352
...,...,...,...,...,...,...,...,...
461018,ags,12:58:19.040,9012,-1399,3879,614,994,-75
461019,ags,12:58:19.045,8873,-1286,3920,578,854,-318
461020,ags,12:58:19.050,9110,-1438,3718,584,787,-373
461021,ags,12:58:19.055,9279,-1206,3920,517,726,-519


,timeHMS,emotion_level,subject
0,0:06:48,4,A
1,0:07:02,4,B
2,0:07:25,1,B
3,0:07:40,4,B
4,0:08:00,2,C
5,0:09:04,1,A
6,0:09:35,1,A
7,0:11:01,4,A
8,0:12:35,-1,C
9,0:13:49,4,B


In [5]:
def norm_order(sensor_data):
    sensor_data["合成加速度"] = np.sqrt(sensor_data["加速度X"]*sensor_data["加速度X"] + sensor_data["加速度Y"]*sensor_data["加速度Y"] + sensor_data["加速度Z"]*sensor_data["加速度Z"])
    sensor_data["合成角速度"] = np.sqrt(sensor_data["角速度X"]*sensor_data["角速度X"] + sensor_data["角速度Y"]*sensor_data["角速度Y"] + sensor_data["角速度Z"]*sensor_data["角速度Z"])
    
norm_order(data1_a)
norm_order(data1_b)
norm_order(data1_c)
norm_order(data1_d)
norm_order(data2_a)
norm_order(data2_b)
norm_order(data2_c)
norm_order(data2_d)

In [6]:
data1_a

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,12:19:53.685,9428,483,4399,163,494,364,10414.973548,634.902355
1,ags,12:19:53.690,9435,463,4382,316,391,407,10413.237633,646.827643
2,ags,12:19:53.695,9449,451,4291,322,263,377,10387.477220,561.232572
3,ags,12:19:53.700,9467,453,4174,181,439,328,10356.233582,577.118705
4,ags,12:19:53.705,9481,309,3989,-25,232,352,10290.625005,422.318600
...,...,...,...,...,...,...,...,...,...,...
461018,ags,12:58:19.040,9012,-1399,3879,614,994,-75,9910.599679,1170.750614
461019,ags,12:58:19.045,8873,-1286,3920,578,854,-318,9785.209502,1079.131132
461020,ags,12:58:19.050,9110,-1438,3718,584,787,-373,9944.016693,1048.596204
461021,ags,12:58:19.055,9279,-1206,3920,517,726,-519,10144.982849,1031.370932


# 3.スケーリングデータに即したセンサデータの抽出
スケーリングデータに即したセンサデータの取得方法は以下の通りである．
1. センサデータ及びスケーリングデータの時刻列をdatetime化
2. timedelta関数を使用して，時刻計算を行い，実験実施日に合わせていく
3. センサデータの時刻列をインデックス化

  時刻列をインデックスにすることで，時間を使ってのデータのスライスが可能となる

4. スケーリングデータの時刻列を文字列型のリストにする
5. ４の操作を行うことでセンサデータからスケーリングデータに即したデータを抽出するのが容易になる

In [7]:
def datetime_order(sensor_data):
    sensor_data['時刻'] = pd.to_datetime(sensor_data['時刻'], format='%H:%M:%S.%f')
    sensor_data['時刻'] = sensor_data['時刻'] + dt.timedelta(days=44144)

datetime_order(data1_a)
datetime_order(data1_b)
datetime_order(data1_c)
datetime_order(data1_d)

datetime_order(data2_a)
datetime_order(data2_b)
datetime_order(data2_c)
datetime_order(data2_d)

In [8]:
#emotion_labelのtimeHMS列をdatetime型に変換
data1_a_label['timeHMS'] = pd.to_datetime(data1_a_label['timeHMS'], format='%M:%S')
data1_b_label['timeHMS'] = pd.to_datetime(data1_b_label['timeHMS'], format='%M:%S')
data1_c_label['timeHMS'] = pd.to_datetime(data1_c_label['timeHMS'], format='%M:%S')
data1_d_label['timeHMS'] = pd.to_datetime(data1_d_label['timeHMS'], format='%H:%M:%S')

data2_a_label['timeHMS'] = pd.to_datetime(data2_a_label['timeHMS'], format='%H:%M:%S')
data2_b_label['timeHMS'] = pd.to_datetime(data2_b_label['timeHMS'], format='%M:%S')
data2_c_label['timeHMS'] = pd.to_datetime(data2_c_label['timeHMS'], format='%H:%M:%S')
data2_d_label['timeHMS'] = pd.to_datetime(data2_d_label['timeHMS'], format='%H:%M:%S')

In [9]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_d)
display(data1_a_label)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,2020-11-11 14:58:06.760,9890,-65,2472,365,610,-105,10194.464625,718.574979
1,ags,2020-11-11 14:58:06.765,9904,37,2450,226,439,-99,10202.601874,503.585147
2,ags,2020-11-11 14:58:06.770,9936,147,2550,73,402,-87,10259.054781,417.734365
3,ags,2020-11-11 14:58:06.775,9929,242,2665,-140,512,-39,10283.279146,532.226456
4,ags,2020-11-11 14:58:06.780,9936,337,2904,-219,292,-21,10357.165684,365.603610
...,...,...,...,...,...,...,...,...,...,...
497905,ags,2020-11-11 15:39:36.525,9367,-63,4931,383,177,95,10585.812156,432.484682
497906,ags,2020-11-11 15:39:36.530,9399,-24,4933,414,110,77,10614.907725,435.229824
497907,ags,2020-11-11 15:39:36.535,9328,59,4970,328,134,-33,10569.577333,355.849687
497908,ags,2020-11-11 15:39:36.540,9201,3,4748,128,463,-172,10353.835714,510.232300


,timeHMS,emotion_level,subject
0,1900-01-01 00:05:47,1,D
1,1900-01-01 00:05:51,1,B
2,1900-01-01 00:05:58,2,C
3,1900-01-01 00:06:07,1,B
4,1900-01-01 00:06:24,0,B
5,1900-01-01 00:06:34,3,D
6,1900-01-01 00:06:57,1,B
7,1900-01-01 00:07:06,3,B
8,1900-01-01 00:07:49,2,C
9,1900-01-01 00:08:29,1,B


### 補足(実験実施時間)

本実験は，zoomレコーディング動画の経過時間でスケーリングデータのemotion_levelをつけたため動画の開始時刻を使って実際の実験実施時間と合わせる必要がある．

方法としては，経過時間と動画の開始時刻を足して実験実施時間を算出する．

動画の開始時間↓

第1セッション12時34分30秒開始

第2セッション15時11分44秒開始

In [10]:
#label_dataを実験日に合わせる(datetime.timedeltaを用いている)
data1_a_label['timeHMS'] = data1_a_label['timeHMS'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_b_label['timeHMS'] = data1_b_label['timeHMS'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_c_label['timeHMS'] = data1_c_label['timeHMS'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_d_label['timeHMS'] = data1_d_label['timeHMS'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

data2_a_label['timeHMS'] = data2_a_label['timeHMS'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_b_label['timeHMS'] = data2_b_label['timeHMS'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_c_label['timeHMS'] = data2_c_label['timeHMS'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_d_label['timeHMS'] = data2_d_label['timeHMS'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [11]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_a_label)

,timeHMS,emotion_level,subject
0,2020-11-11 12:40:17,1,D
1,2020-11-11 12:40:21,1,B
2,2020-11-11 12:40:28,2,C
3,2020-11-11 12:40:37,1,B
4,2020-11-11 12:40:54,0,B
5,2020-11-11 12:41:04,3,D
6,2020-11-11 12:41:27,1,B
7,2020-11-11 12:41:36,3,B
8,2020-11-11 12:42:19,2,C
9,2020-11-11 12:42:59,1,B


In [12]:
#datetime化したラベルデータを保存
data1_a_label.to_csv('emotion_label/label_datetime/emotion_A_20201111_1_datetime.csv', index=False)
data1_b_label.to_csv('emotion_label/label_datetime/emotion_B_20201111_1_datetime.csv', index=False)
data1_c_label.to_csv('emotion_label/label_datetime/emotion_C_20201111_1_datetime.csv', index=False)
data1_d_label.to_csv('emotion_label/label_datetime/emotion_D_20201111_1_datetime.csv', index=False)

data2_a_label.to_csv('emotion_label/label_datetime/emotion_A_20201111_2_datetime.csv', index=False)
data2_b_label.to_csv('emotion_label/label_datetime/emotion_B_20201111_2_datetime.csv', index=False)
data2_c_label.to_csv('emotion_label/label_datetime/emotion_C_20201111_2_datetime.csv', index=False)
data2_d_label.to_csv('emotion_label/label_datetime/emotion_D_20201111_2_datetime.csv', index=False)

In [13]:
#datetime化した加速度及び角速度データを保存
data1_a.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-121953632-ags_datetime.csv', index=False)
data1_b.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-122231707-ags_datetime.csv', index=False)
data1_c.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-121953885-ags_datetime.csv', index=False)
data1_d.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-121954011-ags_datetime.csv', index=False)

data2_a.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-145759193-ags_datetime.csv', index=False)
data2_b.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-145733362-ags_datetime.csv', index=False)
data2_c.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-150018208-ags_datetime.csv', index=False)
data2_d.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-145806707-ags_datetime.csv', index=False)

In [14]:
#時刻をインデックスに
data1_a.set_index("時刻", inplace=True)
data1_b.set_index("時刻", inplace=True)
data1_c.set_index("時刻", inplace=True)
data1_d.set_index("時刻", inplace=True)

data2_a.set_index("時刻", inplace=True)
data2_b.set_index("時刻", inplace=True)
data2_c.set_index("時刻", inplace=True)
data2_d.set_index("時刻", inplace=True)

In [15]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_a)

,種別,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
時刻,,,,,,,,,
2020-11-11 12:19:53.685,ags,9428,483,4399,163,494,364,10414.973548,634.902355
2020-11-11 12:19:53.690,ags,9435,463,4382,316,391,407,10413.237633,646.827643
2020-11-11 12:19:53.695,ags,9449,451,4291,322,263,377,10387.477220,561.232572
2020-11-11 12:19:53.700,ags,9467,453,4174,181,439,328,10356.233582,577.118705
2020-11-11 12:19:53.705,ags,9481,309,3989,-25,232,352,10290.625005,422.318600
...,...,...,...,...,...,...,...,...,...
2020-11-11 12:58:19.040,ags,9012,-1399,3879,614,994,-75,9910.599679,1170.750614
2020-11-11 12:58:19.045,ags,8873,-1286,3920,578,854,-318,9785.209502,1079.131132
2020-11-11 12:58:19.050,ags,9110,-1438,3718,584,787,-373,9944.016693,1048.596204


In [16]:
#label_dataのtimeHMSにおいてdatetime型をstr型にする
data1_a_label["timeHMS"] = data1_a_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_b_label["timeHMS"] = data1_b_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_c_label["timeHMS"] = data1_c_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_d_label["timeHMS"] = data1_d_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")

data2_a_label["timeHMS"] = data2_a_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_b_label["timeHMS"] = data2_b_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_c_label["timeHMS"] = data2_c_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_d_label["timeHMS"] = data2_d_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [17]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_a_label)

,timeHMS,emotion_level,subject
0,2020-11-11 12:40:17,1,D
1,2020-11-11 12:40:21,1,B
2,2020-11-11 12:40:28,2,C
3,2020-11-11 12:40:37,1,B
4,2020-11-11 12:40:54,0,B
5,2020-11-11 12:41:04,3,D
6,2020-11-11 12:41:27,1,B
7,2020-11-11 12:41:36,3,B
8,2020-11-11 12:42:19,2,C
9,2020-11-11 12:42:59,1,B


In [18]:
data1_a_time = list(data1_a_label["timeHMS"])
data1_b_time = list(data1_b_label["timeHMS"])
data1_c_time = list(data1_c_label["timeHMS"])
data1_d_time = list(data1_d_label["timeHMS"])

data2_a_time = list(data2_a_label["timeHMS"])
data2_b_time = list(data2_b_label["timeHMS"])
data2_c_time = list(data2_c_label["timeHMS"])
data2_d_time = list(data2_d_label["timeHMS"])

In [19]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
data1_a_time

['2020-11-11 12:40:17',
 '2020-11-11 12:40:21',
 '2020-11-11 12:40:28',
 '2020-11-11 12:40:37',
 '2020-11-11 12:40:54',
 '2020-11-11 12:41:04',
 '2020-11-11 12:41:27',
 '2020-11-11 12:41:36',
 '2020-11-11 12:42:19',
 '2020-11-11 12:42:59',
 '2020-11-11 12:43:45',
 '2020-11-11 12:44:09',
 '2020-11-11 12:44:39',
 '2020-11-11 12:45:09',
 '2020-11-11 12:45:48',
 '2020-11-11 12:46:50',
 '2020-11-11 12:48:10',
 '2020-11-11 12:49:08',
 '2020-11-11 12:50:03',
 '2020-11-11 12:50:29',
 '2020-11-11 12:50:45',
 '2020-11-11 12:50:54',
 '2020-11-11 12:52:14',
 '2020-11-11 12:52:20',
 '2020-11-11 12:52:26',
 '2020-11-11 12:52:36',
 '2020-11-11 12:53:30',
 '2020-11-11 12:53:55',
 '2020-11-11 12:54:49']

### 抽出時間の定義

In [23]:
def time_plus_order(data_time):
    data_time_plus = pd.to_datetime(data_time)
    data_time_plus = data_time_plus + dt.timedelta(seconds=2)
    data_time_plus = pd.DataFrame(data_time_plus)
    data_time_plus = data_time_plus[0].dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    data_time_plus = list(data_time_plus)
    return data_time_plus

def time_laps_order(data_time):
    data_time_laps = pd.to_datetime(data_time)
    data_time_laps = data_time_laps + dt.timedelta(seconds=6)
    data_time_laps = pd.DataFrame(data_time_laps)
    data_time_laps = data_time_laps[0].dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    data_time_laps = list(data_time_laps)
    return data_time_laps

data1_a_time_plus_list, data1_a_time_laps_list = time_plus_order(data1_a_time), time_laps_order(data1_a_time)
data1_b_time_plus_list, data1_b_time_laps_list = time_plus_order(data1_b_time), time_laps_order(data1_b_time)
data1_c_time_plus_list, data1_c_time_laps_list = time_plus_order(data1_c_time), time_laps_order(data1_c_time)
data1_d_time_plus_list, data1_d_time_laps_list = time_plus_order(data1_d_time), time_laps_order(data1_d_time)

data2_a_time_plus_list, data2_a_time_laps_list = time_plus_order(data2_a_time), time_laps_order(data2_a_time)
data2_b_time_plus_list, data2_b_time_laps_list = time_plus_order(data2_b_time), time_laps_order(data2_b_time)
data2_c_time_plus_list, data2_c_time_laps_list = time_plus_order(data2_c_time), time_laps_order(data2_c_time)
data2_d_time_plus_list, data2_d_time_laps_list = time_plus_order(data2_d_time), time_laps_order(data2_d_time)

In [24]:
def sensor_segmentation_order(data, data_time_plus_list, data_time_laps_list, data_label):
    a = [data[data_time_plus_list[i]:data_time_laps_list[i]]for i in range(0, len(data_label), 1)]
    return a

data1_a_record = sensor_segmentation_order(data1_a, data1_a_time_plus_list, data1_a_time_laps_list, data1_a_label)
data1_b_record = sensor_segmentation_order(data1_b, data1_b_time_plus_list, data1_b_time_laps_list, data1_b_label)
data1_c_record = sensor_segmentation_order(data1_c, data1_c_time_plus_list, data1_c_time_laps_list, data1_c_label)
data1_d_record = sensor_segmentation_order(data1_d, data1_d_time_plus_list, data1_d_time_laps_list, data1_d_label)

data2_a_record = sensor_segmentation_order(data2_a, data2_a_time_plus_list, data2_a_time_laps_list, data2_a_label)
data2_b_record = sensor_segmentation_order(data2_b, data2_b_time_plus_list, data2_b_time_laps_list, data2_b_label)
data2_c_record = sensor_segmentation_order(data2_c, data2_c_time_plus_list, data2_c_time_laps_list, data2_c_label)
data2_d_record = sensor_segmentation_order(data2_d, data2_d_time_plus_list, data2_d_time_laps_list, data2_d_label)

In [25]:
data1_a_record

[                          種別   加速度X  加速度Y  加速度Z  角速度X  角速度Y  角速度Z  \
 時刻                                                                  
 2020-11-11 12:40:19.000  ags  10167   448 -2099    53  -291   102   
 2020-11-11 12:40:19.005  ags  10165   436 -2248    35   -54    72   
 2020-11-11 12:40:19.010  ags  10038   397 -2160    29   141    47   
 2020-11-11 12:40:19.015  ags   9891   446 -2165    59    56    90   
 2020-11-11 12:40:19.020  ags   9818   404 -2028    72  -169    84   
 ...                      ...    ...   ...   ...   ...   ...   ...   
 2020-11-11 12:40:23.975  ags  10663  -291 -1801  1267  -340  -702   
 2020-11-11 12:40:23.980  ags  10641  -199 -1926  1584   -96  -683   
 2020-11-11 12:40:23.985  ags  10573   -67 -2177  1858   177  -623   
 2020-11-11 12:40:23.990  ags  10480   197 -2429  2035   433  -519   
 2020-11-11 12:40:23.995  ags  10367   373 -2583  2066   610  -409   
 
                                 合成加速度        合成角速度  
 時刻                               

# 4.特徴量算出
感情の識別を行うため学習器作成のために特徴量の算出を行う．今回特徴量としたのは，加速度の各軸及び角速度の各軸における時間領域での平均値，分散値，最大値，最小値及び，周波数領域での特徴量である最大ピーク周波数，最大ピーク値である．プログラムの処理手順を以下に示す．

1. 平均値の算出
2. 分散値の算出
3. 最大値の算出
4. 最小値の算出
5. 最大ピーク周波数及び最大ピーク値の算出
5. 全特徴量の数値を表記したデータフレームを作成

## 4.1 平均値算出
### 4.1.1 加速度

In [21]:
#加速度x,y,z,norm平均値
def acc_gyr_mean_order(data_record, data_label):
    a = [np.mean(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.mean(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.mean(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.mean(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_mean_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean"]
#     acc_mean_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean", "accn_mean"]
    e = [np.mean(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.mean(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.mean(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.mean(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_mean_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean"]
#     gyr_mean_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean", "gyrn_mean"]
    mean_list = pd.concat([acc_mean_list, gyr_mean_list], axis=1, ignore_index=False)
    return mean_list

data1_a_record_mean = acc_gyr_mean_order(data1_a_record, data1_a_label)
data1_b_record_mean = acc_gyr_mean_order(data1_b_record, data1_b_label)
data1_c_record_mean = acc_gyr_mean_order(data1_c_record, data1_c_label)
data1_d_record_mean = acc_gyr_mean_order(data1_d_record, data1_d_label)

data2_a_record_mean = acc_gyr_mean_order(data2_a_record, data2_a_label)
data2_b_record_mean = acc_gyr_mean_order(data2_b_record, data2_b_label)
data2_c_record_mean = acc_gyr_mean_order(data2_c_record, data2_c_label)
data2_d_record_mean = acc_gyr_mean_order(data2_d_record, data2_d_label)

# a = data1_a_record_mean
# b = data1_b_record_mean
# c = data1_c_record_mean
# d = data1_d_record_mean

# e = data2_a_record_mean
# f = data2_b_record_mean
# g = data2_c_record_mean
# h = data2_d_record_mean

# mean_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# mean_list

In [22]:
#加速度x,y,z,norm分散値
def acc_gyr_var_order(data_record, data_label):
    a = [np.var(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.var(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.var(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.var(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_var_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_var_list.columns = ["accx_var", "accy_var", "accz_var"]
#     acc_var_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_var_list.columns = ["accx_var", "accy_var", "accz_var", "accn_var"]
    e = [np.var(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.var(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.var(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.var(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_var_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var"]
#     gyr_var_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var", "gyrn_var"]
    var_list = pd.concat([acc_var_list, gyr_var_list], axis=1, ignore_index=False)
    return var_list

data1_a_record_var = acc_gyr_var_order(data1_a_record, data1_a_label)
data1_b_record_var = acc_gyr_var_order(data1_b_record, data1_b_label)
data1_c_record_var = acc_gyr_var_order(data1_c_record, data1_c_label)
data1_d_record_var = acc_gyr_var_order(data1_d_record, data1_d_label)

data2_a_record_var = acc_gyr_var_order(data2_a_record, data2_a_label)
data2_b_record_var = acc_gyr_var_order(data2_b_record, data2_b_label)
data2_c_record_var = acc_gyr_var_order(data2_c_record, data2_c_label)
data2_d_record_var = acc_gyr_var_order(data2_d_record, data2_d_label)

# a = data1_a_record_var
# b = data1_b_record_var
# c = data1_c_record_var
# d = data1_d_record_var

# e = data2_a_record_var
# f = data2_b_record_var
# g = data2_c_record_var
# h = data2_d_record_var

# var_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# var_list

In [23]:
#加速度x,y,z,norm分散値
def acc_gyr_max_order(data_record, data_label):
    a = [np.max(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.max(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.max(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.max(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_max_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_max_list.columns = ["accx_max", "accy_max", "accz_max"]
#     acc_max_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_max_list.columns = ["accx_max", "accy_max", "accz_max", "accn_max"]
    e = [np.max(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.max(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.max(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.max(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_max_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max"]
#     gyr_max_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max", "gyrn_max"]
    max_list = pd.concat([acc_max_list, gyr_max_list], axis=1, ignore_index=False)
    return max_list

data1_a_record_max = acc_gyr_max_order(data1_a_record, data1_a_label)
data1_b_record_max = acc_gyr_max_order(data1_b_record, data1_b_label)
data1_c_record_max = acc_gyr_max_order(data1_c_record, data1_c_label)
data1_d_record_max = acc_gyr_max_order(data1_d_record, data1_d_label)

data2_a_record_max = acc_gyr_max_order(data2_a_record, data2_a_label)
data2_b_record_max = acc_gyr_max_order(data2_b_record, data2_b_label)
data2_c_record_max = acc_gyr_max_order(data2_c_record, data2_c_label)
data2_d_record_max = acc_gyr_max_order(data2_d_record, data2_d_label)

# a = data1_a_record_max
# b = data1_b_record_max
# c = data1_c_record_max
# d = data1_d_record_max

# e = data2_a_record_max
# f = data2_b_record_max
# g = data2_c_record_max
# h = data2_d_record_max

# max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# max_list

In [24]:
#加速度x,y,z,norm分散値
def acc_gyr_min_order(data_record, data_label):
    a = [np.min(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.min(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.min(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.min(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_min_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_min_list.columns = ["accx_min", "accy_min", "accz_min"]
#     acc_min_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_min_list.columns = ["accx_min", "accy_min", "accz_min", "accn_min"]
    e = [np.min(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.min(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.min(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.min(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_min_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min"]
#     gyr_min_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min", "gyrn_min"]
    min_list = pd.concat([acc_min_list, gyr_min_list], axis=1, ignore_index=False)
    return min_list

data1_a_record_min = acc_gyr_min_order(data1_a_record, data1_a_label)
data1_b_record_min = acc_gyr_min_order(data1_b_record, data1_b_label)
data1_c_record_min = acc_gyr_min_order(data1_c_record, data1_c_label)
data1_d_record_min = acc_gyr_min_order(data1_d_record, data1_d_label)

data2_a_record_min = acc_gyr_min_order(data2_a_record, data2_a_label)
data2_b_record_min = acc_gyr_min_order(data2_b_record, data2_b_label)
data2_c_record_min = acc_gyr_min_order(data2_c_record, data2_c_label)
data2_d_record_min = acc_gyr_min_order(data2_d_record, data2_d_label)

# a = data1_a_record_min
# b = data1_b_record_min
# c = data1_c_record_min
# d = data1_d_record_min

# e = data2_a_record_min
# f = data2_b_record_min
# g = data2_c_record_min
# h = data2_d_record_min

# min_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# min_list

## 4.5 最大ピーク周波数及び最大ピーク値

In [25]:
def fq_amp_max_order(data_record, data_label):
    #サンプル数とサンプリング周期及びカットする周波数を設定
    N = 1024 # サンプル数
    dt = 0.005 # サンプリング周期(sec)
    fc = 100
    t = np.arange(0, N*dt, dt) # 時間軸
    
    f = [data_record[i]["合成加速度"].head(1024) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_accn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_accn_fq_amp_max.columns = ["accn_max_fq", "accn_max_peak"]
    
    f = [data_record[i]["合成角速度"].head(1024) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_gyrn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_gyrn_fq_amp_max.columns = ["gyrn_max_fq", "gyrn_max_peak"]
    
    b = pd.concat([data_record_accn_fq_amp_max, data_record_gyrn_fq_amp_max], axis=1, ignore_index=False)
    return b

data1_a_record_fq_amp_max = fq_amp_max_order(data1_a_record, data1_a_label)
data1_b_record_fq_amp_max = fq_amp_max_order(data1_b_record, data1_b_label)
data1_c_record_fq_amp_max = fq_amp_max_order(data1_c_record, data1_c_label)
data1_d_record_fq_amp_max = fq_amp_max_order(data1_d_record, data1_d_label)

data2_a_record_fq_amp_max = fq_amp_max_order(data2_a_record, data2_a_label)
data2_b_record_fq_amp_max = fq_amp_max_order(data2_b_record, data2_b_label)
data2_c_record_fq_amp_max = fq_amp_max_order(data2_c_record, data2_c_label)
data2_d_record_fq_amp_max = fq_amp_max_order(data2_d_record, data2_d_label)

# a = data1_a_record_fq_amp_max
# b = data1_b_record_fq_amp_max
# c = data1_c_record_fq_amp_max
# d = data1_d_record_fq_amp_max

# e = data2_a_record_fq_amp_max
# f = data2_b_record_fq_amp_max
# g = data2_c_record_fq_amp_max
# h = data2_d_record_fq_amp_max

# fq_amp_max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# fq_amp_max_list

In [26]:
#全特徴量の行列の導出
data1_a_record_all_feature = pd.concat([data1_a_record_mean, data1_a_record_var, data1_a_record_max, data1_a_record_min, data1_a_record_fq_amp_max], axis=1, ignore_index=False)
data1_b_record_all_feature = pd.concat([data1_b_record_mean, data1_b_record_var, data1_b_record_max, data1_b_record_min, data1_b_record_fq_amp_max], axis=1, ignore_index=False)
data1_c_record_all_feature = pd.concat([data1_c_record_mean, data1_c_record_var, data1_c_record_max, data1_c_record_min, data1_c_record_fq_amp_max], axis=1, ignore_index=False)
data1_d_record_all_feature = pd.concat([data1_d_record_mean, data1_d_record_var, data1_d_record_max, data1_d_record_min, data1_d_record_fq_amp_max], axis=1, ignore_index=False)

data2_a_record_all_feature = pd.concat([data2_a_record_mean, data2_a_record_var, data2_a_record_max, data2_a_record_min, data2_a_record_fq_amp_max], axis=1, ignore_index=False)
data2_b_record_all_feature = pd.concat([data2_b_record_mean, data2_b_record_var, data2_b_record_max, data2_b_record_min, data2_b_record_fq_amp_max], axis=1, ignore_index=False)
data2_c_record_all_feature = pd.concat([data2_c_record_mean, data2_c_record_var, data2_c_record_max, data2_c_record_min, data2_c_record_fq_amp_max], axis=1, ignore_index=False)
data2_d_record_all_feature = pd.concat([data2_d_record_mean, data2_d_record_var, data2_d_record_max, data2_d_record_min, data2_d_record_fq_amp_max], axis=1, ignore_index=False)

In [27]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
data1_a_record_all_feature

,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,accz_var,gyrx_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,9998.335000,258.181667,-2049.087500,181.728333,156.875000,133.315000,52409.379442,1.746391e+05,1.473336e+05,4.062747e+05,...,9012,-1797,-3188,-1501,-2919,-714,5.474096,105.946434,0.391007,424.369393
1,9988.396147,-1482.352596,-417.927973,99.530988,109.263819,46.846734,56674.292816,1.507672e+06,1.471838e+06,5.114347e+05,...,9012,-3279,-3188,-2116,-2919,-1299,3.519062,68.643399,0.195503,731.844476
2,10006.765833,454.154167,-1875.742500,-172.144167,167.157500,26.727500,97054.629333,2.309472e+05,4.749566e+05,1.379440e+06,...,8636,-991,-5263,-3818,-3474,-1312,4.301075,102.798056,0.195503,1016.143629
3,10050.179167,537.580000,-1673.445833,-2.548333,71.064167,-14.918333,105807.973733,3.295087e+04,3.128601e+05,1.103094e+05,...,8895,-50,-3498,-1122,-4005,-616,3.519062,176.741429,0.195503,401.585970
4,9940.395833,458.375000,-2183.914167,3.536667,139.570833,-21.537500,38400.015816,6.599309e+04,4.496535e+05,1.077478e+05,...,9369,-130,-3471,-1299,-980,-464,5.474096,59.129200,0.195503,193.543279
5,10024.157500,465.665000,-1885.902500,11.180000,33.615000,-24.192500,14046.992694,2.434810e+04,5.583676e+04,2.556129e+04,...,9528,87,-2568,-476,-804,-305,5.474096,67.554886,0.391007,164.132561
6,10050.977500,-449.840000,-731.755000,97.999167,358.979167,74.521667,71181.443660,4.705418e+05,2.614875e+06,3.204263e+05,...,9147,-1589,-3816,-1379,-1346,-568,5.083089,112.589723,0.195503,786.913472
7,10207.465833,-474.429167,199.729167,-67.799167,-86.854167,-123.122500,25507.580499,1.701357e+05,4.122071e+05,1.307799e+05,...,9503,-1162,-2126,-1665,-1566,-1007,5.278592,103.064598,0.195503,291.181454
8,9991.839167,370.172500,-2066.148333,17.594167,31.067500,-19.690833,14029.124966,3.333887e+04,5.668753e+04,8.685692e+04,...,9640,-142,-2773,-677,-1194,-391,5.474096,54.354041,5.669599,86.277245
9,10045.382500,462.569167,-1791.300833,15.155000,36.868333,-19.353333,5291.212860,5.130715e+03,1.008278e+04,1.603922e+04,...,9747,190,-2133,-391,-840,-330,6.451613,28.608530,0.195503,78.073352


In [28]:
#各特徴量データの保存
data1_a_record_all_feature.to_csv('feature/feature_record/data1_a_record_all_feature_kai.csv', index=False)
data1_b_record_all_feature.to_csv('feature/feature_record/data1_b_record_all_feature_kai.csv', index=False)
data1_c_record_all_feature.to_csv('feature/feature_record/data1_c_record_all_feature_kai.csv', index=False)
data1_d_record_all_feature.to_csv('feature/feature_record/data1_d_record_all_feature_kai.csv', index=False)

data2_a_record_all_feature.to_csv('feature/feature_record/data2_a_record_all_feature_kai.csv', index=False)
data2_b_record_all_feature.to_csv('feature/feature_record/data2_b_record_all_feature_kai.csv', index=False)
data2_c_record_all_feature.to_csv('feature/feature_record/data2_c_record_all_feature_kai.csv', index=False)
data2_d_record_all_feature.to_csv('feature/feature_record/data2_d_record_all_feature_kai.csv', index=False)